Johana Tellez-Michael Caicedo
##Descripción general del algoritmo

Los árboles de decisión son modelos supervisados que aprenden reglas jerárquicas para clasificar datos. A partir de los atributos, dividen recursivamente el espacio en regiones más homogéneas, generando reglas del tipo:

“Si player_pos <= 3 y ball_x >= 7, entonces acción = Arriba”

Su ventaja principal es la interpretabilidad, ya que las reglas resultantes pueden leerse como decisiones lógicas. También son rápidos en predicción y pueden generalizar bien si se controlan la profundidad y el tamaño mínimo de hojas.

#1. Definición de PongEnvironment y PongAgent

La clase PongEnvironment define el entorno del juego Pong, incluyendo el tablero, la posición de la pelota y del jugador, las reglas, las recompensas y las penalizaciones. Se encarga de actualizar el estado del juego en cada paso y de detectar colisiones, puntos y finales de partida.

La clase PongAgent representa al agente que aprende a jugar. Contiene la tabla Q, donde se almacenan las políticas aprendidas para cada estado y acción. Además, implementa los métodos para elegir acciones (explorando o explotando) y actualizar la tabla Q usando la ecuación de Bellman en función de las recompensas recibidas.

In [3]:
# ==============================================
# 1. Definición de PongEnvironment y PongAgent
# ==============================================
import numpy as np
import matplotlib.pyplot as plt
from random import randint
from math import ceil, floor
from IPython.display import clear_output

class PongEnvironment:
    def __init__(self, max_life=3, height_px = 40, width_px = 50, movimiento_px = 3):
        self.action_space = ['Arriba','Abajo']
        self._step_penalization = 0
        self.state = [0,0,0]
        self.total_reward = 0
        self.dx = movimiento_px
        self.dy = movimiento_px
        filas = ceil(height_px/movimiento_px)
        columnas = ceil(width_px/movimiento_px)
        self.positions_space = np.array([[[0 for z in range(columnas)]
                                                  for y in range(filas)]
                                                     for x in range(filas)])
        self.lives = max_life
        self.max_life = max_life
        self.x = randint(int(width_px/2), width_px)
        self.y = randint(0, height_px-10)
        self.player_alto = int(height_px/4)
        self.player1 = self.player_alto
        self.score = 0
        self.width_px = width_px
        self.height_px = height_px
        self.radio = 2.5

    def reset(self):
        self.total_reward = 0
        self.state = [0,0,0]
        self.lives = self.max_life
        self.score = 0
        self.x = randint(int(self.width_px/2), self.width_px)
        self.y = randint(0, self.height_px-10)
        return self.state

    def step(self, action, animate=False):
        self._apply_action(action, animate)
        done = self.lives <= 0
        reward = self.score + self._step_penalization
        self.total_reward += reward
        return self.state, reward, done

    def _apply_action(self, action, animate=False):
        if action == "Arriba":
            self.player1 += abs(self.dy)
        elif action == "Abajo":
            self.player1 -= abs(self.dy)
        self.avanza_player()
        self.avanza_frame()
        self.state = (floor(self.player1/abs(self.dy))-2,
                      floor(self.y/abs(self.dy))-2,
                      floor(self.x/abs(self.dx))-2)

    def detectaColision(self, ball_y, player_y):
        return (player_y+self.player_alto >= (ball_y-self.radio)) and (player_y <= (ball_y+self.radio))

    def avanza_player(self):
        if self.player1 + self.player_alto >= self.height_px:
            self.player1 = self.height_px - self.player_alto
        elif self.player1 <= -abs(self.dy):
            self.player1 = -abs(self.dy)

    def avanza_frame(self):
        self.x += self.dx
        self.y += self.dy
        if self.x <= 3 or self.x > self.width_px:
            self.dx = -self.dx
            if self.x <= 3:
                ret = self.detectaColision(self.y, self.player1)
                if ret:
                    self.score = 10
                else:
                    self.score = -10
                    self.lives -= 1
                    if self.lives > 0:
                        self.x = randint(int(self.width_px/2), self.width_px)
                        self.y = randint(0, self.height_px-10)
                        self.dx = abs(self.dx)
                        self.dy = abs(self.dy)
        else:
            self.score = 0
        if self.y < 0 or self.y > self.height_px:
            self.dy = -self.dy

class PongAgent:
    def __init__(self, game, policy=None, discount_factor = 0.1, learning_rate = 0.1, ratio_explotacion = 0.9):
        if policy is not None:
            self._q_table = policy
        else:
            position = list(game.positions_space.shape)
            position.append(len(game.action_space))
            self._q_table = np.zeros(position)
        self.discount_factor = discount_factor
        self.learning_rate = learning_rate
        self.ratio_explotacion = ratio_explotacion

    def get_next_step(self, state, game):
        next_step = np.random.choice(list(game.action_space))
        if np.random.uniform() <= self.ratio_explotacion:
            idx_action = np.random.choice(np.flatnonzero(
                self._q_table[state[0],state[1],state[2]] ==
                self._q_table[state[0],state[1],state[2]].max()
            ))
            next_step = list(game.action_space)[idx_action]
        return next_step

    def update(self, game, old_state, action_taken, reward_action_taken, new_state, reached_end):
        idx_action_taken = list(game.action_space).index(action_taken)
        actual_q_value = self._q_table[old_state[0], old_state[1], old_state[2], idx_action_taken]
        future_max_q_value = reward_action_taken + self.discount_factor * self._q_table[new_state[0], new_state[1], new_state[2]].max()
        if reached_end:
            future_max_q_value = reward_action_taken
        self._q_table[old_state[0], old_state[1], old_state[2], idx_action_taken] = actual_q_value + self.learning_rate*(future_max_q_value - actual_q_value)

    def get_policy(self):
        return self._q_table



##2. Función de entrenamiento
La función de entrenamiento (play) se encarga de simular múltiples partidas de Pong para que el agente aprenda por refuerzo. Durante cada episodio, el agente interactúa con el entorno, elige acciones, recibe recompensas y actualiza su tabla Q para mejorar sus decisiones futuras.

La función controla el número de iteraciones, acumula estadísticas como puntajes y pasos promedio, y ajusta las políticas del agente con base en la experiencia acumulada, permitiendo que este evolucione desde un comportamiento aleatorio hasta una estrategia más eficiente.

In [4]:
# ============================
# 2. Función de entrenamiento
# ============================
def play(rounds=5000, max_life=3, discount_factor = 0.1, learning_rate = 0.1, ratio_explotacion=0.9, learner=None, game=None):
    if game is None:
        game = PongEnvironment(max_life=max_life, movimiento_px=3)
    if learner is None:
        learner = PongAgent(game, discount_factor=discount_factor, learning_rate=learning_rate, ratio_explotacion=ratio_explotacion)

    for _ in range(rounds):
        state = game.reset()
        done = False
        while not done:
            old_state = np.array(state)
            action = learner.get_next_step(state, game)
            state, reward, done = game.step(action)
            learner.update(game, old_state, action, reward, state, done)
    return learner, game



## 3. Entrenamiento + Árbol de decisión
En esta etapa se utiliza la tabla Q entrenada por el agente para construir un dataset supervisado, donde cada estado del juego (posición del jugador y de la pelota) actúa como entrada, y la mejor acción aprendida corresponde a la etiqueta.

Con este dataset, se entrena un modelo de Árbol de Decisión para que aprenda a imitar la política del agente. Luego, el modelo es evaluado en un conjunto de validación independiente, calculando métricas como precisión y recall. Esto permite verificar qué tan bien el árbol logra reproducir las decisiones del agente, facilitando su uso posterior sin necesidad de ejecutar Q-Learning en tiempo real.

In [5]:
# ===============================
# 3. Entrenamiento + Árbol de decisión
# ===============================
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

# Entrenar agente y obtener política
learner, game = play(rounds=5000)

q_table = learner.get_policy()
pos_n, y_n, x_n, n_actions = q_table.shape

# Dataset
X, y = [], []
for i in range(pos_n):
    for j in range(y_n):
        for k in range(x_n):
            best_action_idx = int(np.argmax(q_table[i, j, k]))
            X.append([i, j, k])
            y.append(best_action_idx)

X = np.array(X)
y = np.array(y)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Árbol de decisión
tree_model = DecisionTreeClassifier(random_state=42)
tree_model.fit(X_train, y_train)

y_pred = tree_model.predict(X_val)
print("Exactitud Árbol de Decisión:", accuracy_score(y_val, y_pred))
print(classification_report(y_val, y_pred))


Exactitud Árbol de Decisión: 0.6626686656671664
              precision    recall  f1-score   support

           0       0.70      0.71      0.70       378
           1       0.61      0.61      0.61       289

    accuracy                           0.66       667
   macro avg       0.66      0.66      0.66       667
weighted avg       0.66      0.66      0.66       667



##Intrepretación
El modelo de árbol de decisión obtuvo una exactitud del 66,3 %, lo que indica que aproximadamente dos de cada tres predicciones coinciden con las acciones reales definidas en la tabla Q.

Para la clase 0 (acción “Arriba”), se logró una precisión de 0.70 y un recall de 0.71, mostrando un buen desempeño en la identificación de esta acción. En la clase 1 (acción “Abajo”), la precisión y el recall son de 0.61, lo que revela un comportamiento más modesto, posiblemente por diferencias en la distribución de ejemplos o mayor complejidad en la toma de decisiones para esa clase.

En general, el árbol logra capturar correctamente la estrategia del agente en la mayoría de los casos, mostrando un balance razonable entre ambas clases. Sin embargo, aún existe espacio para mejorar mediante ajustes en los hiperparámetros o técnicas de optimización para afinar la generalización del modelo.